In [ ]:
import getpass
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from gtts import gTTS
from IPython.display import Audio, display

In [ ]:
# Get API keys
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt="Enter your OpenAI API key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass(prompt="Enter your LangChain API key: ")

# Load dataset
file_path = r'C:\Users\Haris\Documents\Software Questions.csv'
df = pd.read_csv(file_path, encoding='cp1252')

# Convert questions to a list of texts
docs = df['Question'].tolist()

In [ ]:
# Create vector store and retriever
vectorstore = Chroma.from_texts(texts=docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# Initialize language model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
# Define prompts
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

system_prompt = (
    "You are a job interviewer related to computer science and software engineering. Take the user's interview."
    "Use the following pieces of retrieved context to answer "
    "the question and take user's interview. Ask at least 5 questions, one by one, and then evaluate the user yourself "
    "whether is the user able for the position or not."
    "If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create chains
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
# Store for session history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Combine retrieval chain with message history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Define TTS function
def speak_text(text):
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

In [ ]:
# Import speech recognition library
import speech_recognition as sr

# Define speech-to-text function
def listen_to_user():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
    try:
        print("Recognizing...")
        user_input = r.recognize_google(audio)
        print("You:", user_input)
        return user_input
    except sr.UnknownValueError:
        print("Could not understand audio")
        return ""
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
        return ""

# Example usage within your chatbot loop
while True:
    user_input = listen_to_user()
    if user_input.lower() == "quit":
        print("Goodbye!")
        break
    response = conversational_rag_chain.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": "abc123"}},
    )["answer"]
    print("Bot:", response)
    speak_text(response)